In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3435438/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
1,model_1_7_1,0.946189,0.830081,0.618629,0.799365,0.059552,0.191661,0.094949,0.146150,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
3,model_1_4_1,0.931944,0.620239,0.891248,0.809584,0.075318,0.174556,0.119141,0.148478,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
6,model_1_7_0,0.925489,0.859492,0.643891,0.827538,0.082462,0.158487,0.088660,0.125627,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
7,model_1_4_2,0.924933,0.620635,0.804482,0.752342,0.083077,0.174374,0.214194,0.193113,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
10,model_1_4_0,0.919022,0.618529,0.928851,0.833912,0.089619,0.175342,0.077945,0.129508,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
...,...,...,...,...,...,...,...,...,...,...
186,model_4_3_18,0.863687,0.482388,0.852034,0.771339,0.150859,0.372341,0.279112,0.328469,"Hidden Size=[40, 20], regularizer=0.02, learni..."
188,model_4_8_3,0.863520,0.658140,0.990199,0.816721,0.151043,0.315331,0.009271,0.171303,"Hidden Size=[40, 20], regularizer=0.02, learni..."
189,model_4_3_17,0.863184,0.491062,0.854150,0.774946,0.151415,0.366101,0.275121,0.323287,"Hidden Size=[40, 20], regularizer=0.02, learni..."
192,model_4_8_12,0.862557,0.460284,0.970219,0.703831,0.152109,0.497834,0.028172,0.276817,"Hidden Size=[40, 20], regularizer=0.02, learni..."
